In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # works with and without this 

import torch
from peft import PeftModelForSeq2SeqLM, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


peft_model_id = "lukaemon/flan-t5-xxl-financial-phrasebank-lora"

peft_config = PeftConfig.from_pretrained(peft_model_id)

peft_config.base_model_name_or_path


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


'google/flan-t5-xxl'

In [2]:
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

model = AutoModelForSeq2SeqLM.from_pretrained(peft_config.base_model_name_or_path, device_map='auto', load_in_8bit=True)

model = PeftModelForSeq2SeqLM.from_pretrained(model, peft_model_id, device_map={"": 0})

model.print_trainable_parameters()
model.hf_device_map

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

trainable params: 18874368 || all params: 11154206720 || trainable%: 0.16921300163961817


{'base_model.model.shared': 0,
 'base_model.model.decoder.embed_tokens': 0,
 'base_model.model.encoder.embed_tokens': 0,
 'base_model.model.encoder.block.0': 0,
 'base_model.model.encoder.block.1': 0,
 'base_model.model.encoder.block.2': 0,
 'base_model.model.encoder.block.3': 0,
 'base_model.model.encoder.block.4': 0,
 'base_model.model.encoder.block.5': 0,
 'base_model.model.encoder.block.6': 0,
 'base_model.model.encoder.block.7': 0,
 'base_model.model.encoder.block.8': 0,
 'base_model.model.encoder.block.9': 0,
 'base_model.model.encoder.block.10': 0,
 'base_model.model.encoder.block.11': 0,
 'base_model.model.encoder.block.12': 0,
 'base_model.model.encoder.block.13': 0,
 'base_model.model.encoder.block.14': 0,
 'base_model.model.encoder.block.15': 0,
 'base_model.model.encoder.block.16': 0,
 'base_model.model.encoder.block.17': 0,
 'base_model.model.encoder.block.18': 'cpu',
 'base_model.model.encoder.block.19': 'cpu',
 'base_model.model.encoder.block.20': 'cpu',
 'base_model.mod

In [3]:
input_text = "In January-September 2009 , the Group 's net interest income increased to EUR 112.4 mn from EUR 74.3 mn in January-September 2008 ."
inputs = tokenizer(input_text, return_tensors="pt")

# with torch.cuda.amp.autocast():    
outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=10)

print("input sentence: ", input_text)
print("output prediction: ", tokenizer.decode(outputs[0], skip_special_tokens=True))


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   2 inputs = tokenizer(input_text, return_tensors="pt")                                          │
│   3                                                                                              │
│   4 # with torch.cuda.amp.autocast():                                                            │
│ ❱ 5 outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=10)                   │
│   6                                                                                              │
│   7 print("input sentence: ", input_text)                                                        │
│   8 print("output prediction: ", tokenizer.decode(outputs[0], skip_special_tokens=True))         │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/peft/peft_model.py:728 in generate                        │
│                                                                                                  │
│   725 │                                                                                          │
│   726 │   def generate(self, **kwargs):                                                          │
│   727 │   │   if not isinstance(self.peft_config, PromptLearningConfig):                         │
│ ❱ 728 │   │   │   return self.base_model.generate(**kwargs)                                      │
│   729 │   │   else:                                                                              │
│   730 │   │   │   if "input_ids" not in kwargs:                                                  │
│   731 │   │   │   │   raise ValueError("input_ids must be provided for Peft model generation")   │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/autograd/grad_mode.py:27 in decorate_context        │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1264 in generate         │
│                                                                                                  │
│   1261 │   │   if self.config.is_encoder_decoder and "encoder_outputs" not in model_kwargs:      │
│   1262 │   │   │   # if model is encoder decoder encoder_outputs are created                     │
│   1263 │   │   │   # and added to `model_kwargs`                                                 │
│ ❱ 1264 │   │   │   model_kwargs = self._prepare_encoder_decoder_kwargs_for_generation(           │
│   1265 │   │   │   │   inputs_tensor, model_kwargs, model_input_name                             │
│   1266 │   │   │   )                                                                             │
│   1267                                                     